In [1]:
# set module
import pandas as pd
from google.colab import drive
drive.mount('/content/drive/')
#경로 설정
import os
new_dir = "/content/drive/MyDrive/기상청/데이터/지면온도"
os.chdir(new_dir)  # 현재 작업 디렉토리 변경
current_dir = os.getcwd()
print("현재 작업 디렉토리:", current_dir)

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).
현재 작업 디렉토리: /content/drive/.shortcut-targets-by-id/1RsCY7hPEjVtDTeOTNakzD06_onT7ryDd/기상청/데이터/지면온도


In [2]:
# 데이터 읽어오기
data_train = pd.read_csv("./surface_tp_train.csv")
data_test= pd.read_csv("./surface_tp_test.csv")

In [3]:
## column 명 추출
data_train.drop('Unnamed: 0', axis = 1, inplace = True)
data_train.drop('...1', axis = 1, inplace = True)
data_test.drop('Unnamed: 0', axis = 1, inplace = True)

## 시각화 진행

In [4]:
## 대푯값 확인 -> train
data_train.describe()

,surface_tp_train.stn,surface_tp_train.mmddhh,surface_tp_train.ta,surface_tp_train.td,surface_tp_train.hm,surface_tp_train.ws,surface_tp_train.rn,surface_tp_train.re,surface_tp_train.ts,surface_tp_train.si,surface_tp_train.ss,surface_tp_train.sn
count,438240.000000,438240.000000,438240.000000,438240.000000,438240.000000,438240.000000,438240.000000,438240.000000,438240.000000,438240.000000,438240.000000,438240.000000
mean,5.500000,66819.769441,13.648830,6.546744,65.429143,1.982703,-1.539624,2.608285,15.268847,-45.005243,-45.094056,-98.205804
std,2.872285,34506.091290,11.042725,12.627285,22.430588,5.156003,12.996733,13.570768,13.362203,50.307733,50.000381,13.128373
min,1.000000,10100.000000,-99.900000,-99.900000,-99.900000,-99.900000,-99.900000,-99.000000,-99.900000,-99.900000,-99.900000,-99.900000
25%,3.000000,40202.000000,5.800000,-2.400000,49.200000,1.000000,0.000000,0.000000,4.800000,-99.900000,-99.900000,-99.900000
50%,5.500000,70209.000000,14.700000,7.300000,66.500000,1.800000,0.000000,0.000000,15.500000,0.010000,0.000000,-99.900000
75%,8.000000,100116.000000,22.200000,17.100000,83.900000,3.000000,0.000000,0.000000,24.300000,0.990000,0.700000,-99.900000
max,10.000000,123123.000000,39.400000,29.300000,100.000000,24.600000,623.500000,60.000000,65.700000,4.770000,1.000000,37.400000


In [5]:
data_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 438240 entries, 0 to 438239
Data columns (total 14 columns):
 #   Column                   Non-Null Count   Dtype  
---  ------                   --------------   -----  
 0   surface_tp_train.stn     438240 non-null  int64  
 1   surface_tp_train.year    438240 non-null  object 
 2   surface_tp_train.mmddhh  438240 non-null  int64  
 3   surface_tp_train.ta      438240 non-null  float64
 4   surface_tp_train.td      438240 non-null  float64
 5   surface_tp_train.hm      438240 non-null  float64
 6   surface_tp_train.ws      438240 non-null  float64
 7   surface_tp_train.rn      438240 non-null  float64
 8   surface_tp_train.re      438240 non-null  int64  
 9   surface_tp_train.ww      438240 non-null  object 
 10  surface_tp_train.ts      438240 non-null  float64
 11  surface_tp_train.si      438240 non-null  float64
 12  surface_tp_train.ss      438240 non-null  float64
 13  surface_tp_train.sn      438240 non-null  float64
dtypes: f

In [6]:
## 제거 : -99, -99.9, -999

import pandas as pd
import numpy as np

class data_preprocessing():
  def __init__(self, data, test):
    self.data = data
    self.na_counts = None
    self.list_na = None
    self.ratio_result = None
    self.list_columns = None
    self.test = test

  ## na 개수 발견
  def count_na_per_column(self):
    special_values = [-99, -99.9, -999]
    list_counts = []
    for col in self.data.columns:
        counts = self.data[col].isin(special_values).sum()
        list_counts.append(counts)
    self.list_na = list_counts
    return self.list_na

  def divide_list_elements(self, lst, divisor):
      divided_lst = [x / divisor for x in lst]
      return divided_lst

  ## na 개수 전체 데이터 중 비율 세기
  def ratio(self):
    self.ratio_result = self.divide_list_elements(self.list_na, len(self.data))
    return self.ratio_result

  def remove_column(self):
    self.list_columns = []
    # 역순으로 반복문 실행하여 열을 삭제
    for i in range(len(self.ratio_result)-1, -1, -1):
        if self.ratio_result[i] >= 0.4:
          self.list_columns.append(self.data.columns[i])
            # print(self.ratio_result[i])
            # print(i)
          self.data = self.data.drop(self.data.columns[i], axis=1)
    return self.data

  def remove_columns_test(self):
    data = self.data
    for i in self.list_columns:
      ## . 기준으로 마지막 부분 없애고,
      new_col = i.replace("train", "test")
      # print(new_col)
      try: ## train만 있는 열이 있을 경우에는 pass 하자
        data = data.drop(new_col, axis=1)
      except:
        pass
    return data


  def result(self, train_or_test):
    if train_or_test == "train":
      # self.replace_99()
      self.count_na_per_column()
      self.ratio()
      return self.remove_column()
    else:
      self.data = self.test
      # self.replace_99() ## null 로 변경
      return self.remove_columns_test() ## train에서 제거되는 행으로 제거

In [7]:
dp= data_preprocessing(data_train, data_test)
result_train = dp.result('train')
result_train
result_test = dp.result("test")
result_test

,surface_tp_test.stn,surface_tp_test.year,surface_tp_test.mmddhh,surface_tp_test.ta,surface_tp_test.td,surface_tp_test.hm,surface_tp_test.ws,surface_tp_test.rn,surface_tp_test.re,surface_tp_test.ww
0,a,F,20100,0.6,-2.0,82.5,2.7,0.0,0,G
1,a,F,20101,0.0,-5.2,68.3,3.2,0.0,0,R
2,a,F,20102,-0.3,-6.4,63.7,2.7,0.0,0,C
3,a,F,20103,-1.0,-4.5,77.2,2.1,0.2,7,R
4,a,F,20104,-1.4,-3.1,88.3,2.9,0.6,3,R
...,...,...,...,...,...,...,...,...,...,...
26275,c,G,13119,7.1,-1.9,53.0,5.7,0.0,0,C
26276,c,G,13120,6.7,-0.5,60.1,4.7,0.0,0,C
26277,c,G,13121,6.2,-0.1,63.9,3.7,0.0,0,C
26278,c,G,13122,6.5,0.8,67.1,4.8,0.0,0,C


In [8]:
result_train.dropna(inplace = True)
result_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 438240 entries, 0 to 438239
Data columns (total 11 columns):
 #   Column                   Non-Null Count   Dtype  
---  ------                   --------------   -----  
 0   surface_tp_train.stn     438240 non-null  int64  
 1   surface_tp_train.year    438240 non-null  object 
 2   surface_tp_train.mmddhh  438240 non-null  int64  
 3   surface_tp_train.ta      438240 non-null  float64
 4   surface_tp_train.td      438240 non-null  float64
 5   surface_tp_train.hm      438240 non-null  float64
 6   surface_tp_train.ws      438240 non-null  float64
 7   surface_tp_train.rn      438240 non-null  float64
 8   surface_tp_train.re      438240 non-null  int64  
 9   surface_tp_train.ww      438240 non-null  object 
 10  surface_tp_train.ts      438240 non-null  float64
dtypes: float64(6), int64(3), object(2)
memory usage: 36.8+ MB


In [9]:
# result_train['hh'] = result_train['surface_tp_train.mmddhh'].astype(int).astype(str).str[-2:]
# result_train['dd'] = result_train['surface_tp_train.mmddhh'].astype(int).astype(str).str[-4:-2]
# result_train['mm'] = result_train['surface_tp_train.mmddhh'].astype(int).astype(str).str[:-4]
# year_mapping = {'A': 2001, 'B': 2002, 'C': 2003, 'D': 2004, 'E': 2005, 'F': 2006}
# result_train['new_year'] = result_train['surface_tp_train.year'].map(year_mapping)
# result_train['mm'] = result_train['mm'].apply(lambda x: x.zfill(2))
# result_train['date'] = pd.to_datetime(result_train['new_year'].astype(str) + result_train['mm'] + result_train['dd'] + result_train['hh'], format='%Y%m%d%H')

def adding_date(data_original, season): # season: all, spring, summer, fall, winter 계절별로 넣기 가능
  data = data_original
  data['hh'] = data['surface_tp_train.mmddhh'].astype(int).astype(str).str[-2:]
  data['dd'] = data['surface_tp_train.mmddhh'].astype(int).astype(str).str[-4:-2]
  data['mm'] = data['surface_tp_train.mmddhh'].astype(int).astype(str).str[:-4]
  year_mapping = {'A': 2001, 'B': 2002, 'C': 2003, 'D': 2004, 'E': 2005, 'F': 2006}
  data['new_year'] = data['surface_tp_train.year'].map(year_mapping)
  data['mm'] = data['mm'].apply(lambda x: x.zfill(2))
  data['date'] = pd.to_datetime(data['new_year'].astype(str) + data['mm'] + data['dd'] + data['hh'], format='%Y%m%d%H')

  spring_month_list = ['02', '03', '04']
  summer_month_list = ['05', '06', '07']
  fall_month_list = ['08', '09', '10']
  winter_month_list = ['11', '12', '01']

  if season == "all":
    pass
  elif season == "spring":
    filtered_data = data[data['mm'].isin(spring_month_list)]
  elif season == "summer":
    filtered_data = data[data['mm'].isin(summer_month_list)]
  elif season == "fall":
    filtered_data = data[data['mm'].isin(fall_month_list)]
  elif season == "winter":
    filtered_data = data[data['mm'].isin(winter_month_list)]
  else:
    raise ValueError("not season value")

  # data = filtered_data.drop(["surface_tp_train.year", 'surface_tp_train.mmddhh', 'hh', 'dd', 'mm', 'new_year'], axis = 1)
  data = filtered_data.drop(["surface_tp_train.year", 'surface_tp_train.mmddhh'], axis = 1)
  # data["datetime"] = pd.to_datetime(data["date"])  # 문자열을 날짜/시간 유형으로 변환
  ## datetime 순으로 index 생성
  data.reset_index(inplace=True)
  data.index = data.index.astype(int)
  return data

### 지역 별로 데이터를 쪼개주는 함수

def region_category(data, region_number): # region_number는 자기가 알아서 넣기로

  # 주어진 열을 기준으로 데이터 프레임을 필터링하고 값 종류별로 데이터를 쪼갬
  unique_values = data['surface_tp_train.stn'].unique()  # 주어진 열의 고유한 값들을 추출

  if region_number in unique_values:
    result = data[data['surface_tp_train.stn'] == region_number]  # 주어진 값에 해당하는 데이터만 필터링하여 저장
  else:
    raise ValueError("not in region number")

  return result


def change_columns(data):
  data.columns = data.columns.str.replace(".", "_")


def change_to_category(data, columns_name):
  data[columns_name] = data[columns_name].astype(str).astype('category')
  return data


In [10]:
result_train_spring = adding_date(result_train, "spring")
result_train_spring.columns

## region 별로 나누기
train_region_1_spring = region_category(result_train_spring, 1)
train_region_2_spring = region_category(result_train_spring, 2)
train_region_3_spring = region_category(result_train_spring, 3)
train_region_4_spring = region_category(result_train_spring, 4)
train_region_5_spring = region_category(result_train_spring, 5)
train_region_6_spring = region_category(result_train_spring, 6)
train_region_7_spring = region_category(result_train_spring, 7)
train_region_8_spring = region_category(result_train_spring, 8)
train_region_9_spring = region_category(result_train_spring, 8)
train_region_10_spring = region_category(result_train_spring, 10)

In [11]:
## category 명 변경;
def category_changing(data):
  list_columns = ["surface_tp_train_stn", "new_year", "mm", "dd", "hh", "surface_tp_train_ww"]
  for column in list_columns:
      data = change_to_category(data, column)

## time index  제작:
def indexing(data):
  ## column 이름에 . 제거
  change_columns(data)
  data.drop("index", axis = 1, inplace=True)
  ## sort 하고 타임 index 생성
  data.sort_values("date", ascending = True, inplace = True)
  data['time_idx'] = range(len(data))
  return data

## date drop:
def drop_date(data):
  data.drop("date", axis = 1, inplace = True)

## main 실행
list_data = [train_region_1_spring, train_region_2_spring, train_region_3_spring, train_region_4_spring, train_region_5_spring
             ,train_region_6_spring,train_region_7_spring,train_region_8_spring,train_region_9_spring, train_region_10_spring]

for data in list_data:
  change_columns(data)
  category_changing(data)
  data = indexing(data)
  drop_date(data)

<ipython-input-9-bc2a47df1858>:61: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  data.columns = data.columns.str.replace(".", "_")
<ipython-input-9-bc2a47df1858>:65: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data[columns_name] = data[columns_name].astype(str).astype('category')
<ipython-input-9-bc2a47df1858>:65: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus

In [12]:
new_directory = "/content/drive/MyDrive/기상청 (1)"
os.chdir(new_directory)
root_directory = os.getcwd()
root_directory

'/content/drive/MyDrive/기상청 (1)'

In [13]:
train_region_1_spring

,surface_tp_train_stn,surface_tp_train_ta,surface_tp_train_td,surface_tp_train_hm,surface_tp_train_ws,surface_tp_train_rn,surface_tp_train_re,surface_tp_train_ww,surface_tp_train_ts,hh,dd,mm,new_year,time_idx
0,1,-9.9,-10.7,93.9,0.6,0.0,0,H,-1.3,00,01,02,2001,0
1,1,-10.8,-11.6,93.8,0.6,0.0,0,H,-1.5,01,01,02,2001,1
2,1,-11.4,-12.1,94.6,0.7,0.0,0,H,-1.7,02,01,02,2001,2
3,1,-11.6,-12.5,93.4,0.6,0.0,0,H,-1.8,03,01,02,2001,3
4,1,-11.8,-12.7,93.0,0.6,0.0,0,H,-2.0,04,01,02,2001,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10699,1,13.1,8.5,73.9,1.1,0.0,0,C,14.4,19,30,04,2005,10699
10700,1,12.8,8.3,74.3,1.2,0.0,0,C,13.4,20,30,04,2005,10700
10701,1,12.2,9.4,82.8,0.7,0.0,0,C,12.7,21,30,04,2005,10701
10702,1,11.9,9.8,87.0,1.0,0.0,0,H,12.8,22,30,04,2005,10702


In [14]:
!nvidia-smi

Wed Jun 28 18:14:51 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   42C    P8     9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [15]:
import torch, gc
gc.collect()
torch.cuda.empty_cache()

## LSTM 코드 구현

### 변수
<div>
  <table>
    <tr>
      <th>Column</th>
      <th>Description</th>
      <th>Data Type</th>
    </tr>
    <tr>
      <td>surface_tp_train.stn</td>
      <td>지역번호</td>
      <td>category</td>
    </tr>
    <tr>
      <td>surface_tp_train.ta</td>
      <td>1시간 평균 기온</td>
      <td>numeric</td>
    </tr>
    <tr>
      <td>surface_tp_train.td</td>
      <td>1시간 평균 이슬점온도</td>
      <td>numeric</td>
    </tr>
    <tr>
      <td>surface_tp_train.hm</td>
      <td>1시간 평균 상대 습도</td>
      <td>numeric</td>
    </tr>
    <tr>
      <td>surface_tp_train.ws</td>
      <td>1시간 평균 풍속</td>
      <td>numeric</td>
    </tr>
    <tr>
      <td>surface_tp_train.rn</td>
      <td>1시간 누적 강수량</td>
      <td>numeric</td>
    </tr>
    <tr>
      <td>surface_tp_train.re</td>
      <td>1시간 누적 강수 유무</td>
      <td>category</td>
    </tr>
    <tr>
      <td>surface_tp_train.ww</td>
      <td>현천개 현천</td>
      <td>category</td>
    </tr>
    <tr>
      <td>surface_tp_train.ts</td>
      <td>1시간 평균 지면온도</td>
      <td>target</td>
    </tr>
    <tr>
      <td>hh</td>
      <td>시간</td>
      <td>category</td>
    </tr>
    <tr>
      <td>dd</td>
      <td>날짜</td>
      <td>category</td>
    </tr>
    <tr>
      <td>mm</td>
      <td>달</td>
      <td>category</td>
    </tr>
    <tr>
      <td>new_year</td>
      <td>연도</td>
      <td>category</td>
    </tr>
    <tr>
      <td>date</td>
      <td>총 시간 datetime</td>
      <td>category</td>
    </tr>
  </table>
</div>


In [16]:
## 지역 개수 10개를 병렬적으로 학습하게 넣을 예정
## window size 지정 필요

## 들어가는 데이터 열의 개수는

"""
'surface_tp_train_stn', 'surface_tp_train_ta', 'surface_tp_train_td',
       'surface_tp_train_hm', 'surface_tp_train_ws', 'surface_tp_train_rn',
       'surface_tp_train_re', 'surface_tp_train_ww', 'surface_tp_train_ts','time_idx'
       에서 target값인 surface_tp_train_ts, groupby pivot인 surface_tp_train_stn 를 제거하고 넣을 예정
       :8가지
"""
## model import
import numpy as np
import pandas as pd
import pandas_datareader.data as pdr
import matplotlib.pyplot as plt
import datetime
import torch
import torch.nn as nn
from torch.autograd import Variable
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

# GPU setting
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(torch.cuda.get_device_name(0)) #Google colab = TESLA T4

# Training/Test ratio = 8:2

Train_ratio = 0.8
Test_ratio = 0.2

def labelencoding(data):
  x = data[['surface_tp_train_stn', 'surface_tp_train_ta', 'surface_tp_train_td',
        'surface_tp_train_hm', 'surface_tp_train_ws', 'surface_tp_train_rn',
        'surface_tp_train_re', 'surface_tp_train_ww', 'time_idx']]
  y = data["surface_tp_train_ts"]


  ## x에서 문자열 라벨 인코딩 진행:
  le = LabelEncoder()
  le.fit(x.surface_tp_train_ww)
  x.surface_tp_train_ww = le.transform(x.surface_tp_train_ww)
  return x, y

x_region_1_spring, y_region_1_spring = labelencoding(train_region_1_spring)
x_region_2_spring, y_region_2_spring = labelencoding(train_region_2_spring)
x_region_3_spring, y_region_3_spring = labelencoding(train_region_3_spring)
x_region_4_spring, y_region_4_spring = labelencoding(train_region_4_spring)
x_region_5_spring, y_region_5_spring = labelencoding(train_region_5_spring)
x_region_6_spring, y_region_6_spring = labelencoding(train_region_6_spring)
x_region_7_spring, y_region_7_spring = labelencoding(train_region_7_spring)
x_region_8_spring, y_region_8_spring = labelencoding(train_region_8_spring)
x_region_9_spring, y_region_9_spring = labelencoding(train_region_9_spring)
x_region_10_spring, y_region_10_spring = labelencoding(train_region_10_spring)

Tesla T4


<ipython-input-16-54df200d359c>:47: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  x.surface_tp_train_ww = le.transform(x.surface_tp_train_ww)
<ipython-input-16-54df200d359c>:47: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  x.surface_tp_train_ww = le.transform(x.surface_tp_train_ww)
<ipython-input-16-54df200d359c>:47: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https

In [17]:
def before_modeling(x, y, scaler = 'minmax'):
  # Data Division
  train_x, test_x, train_y, test_y = train_test_split(x, y, test_size=Test_ratio, random_state=42, shuffle = False)

  ## scaling:
  # 카테고리 변수 지정
  cat = ["surface_tp_train_re", "surface_tp_train_ww", "surface_tp_train_stn", "time_idx"]

  for category in cat:
    train_x[category] = train_x[category].astype("category")

  numeric_cols = [col for col in train_x.columns if col not in cat]

  if scaler == 'minmax':
  # Normalizing , 둘다 학습하여 성능비교할것
    minmax = MinMaxScaler()
    train_y = train_y.values.reshape(-1, 1)
    test_y = test_y.values.reshape(-1, 1)
    train_x[numeric_cols]= minmax.fit_transform(train_x[numeric_cols])
    test_x[numeric_cols] = minmax.fit_transform(test_x[numeric_cols])
  elif scaler == 'standard':
    standard = StandardScaler()
    train_y = train_y.values.reshape(-1, 1)
    test_y = test_y.values.reshape(-1, 1)
    train_x[numeric_cols]= standard.fit_transform(train_x[numeric_cols])
    test_x[numeric_cols] = standard.fit_transform(test_x[numeric_cols])

  # Check Data pre-processing
  print("Training shape : ", train_x.shape, train_y.shape)
  print("Test shape : ",test_x.shape, test_y.shape)
  return train_x, test_x, train_y, test_y

In [18]:
## minmax scaling 진행
train_x_region_1_spring, test_x_region_1_spring, train_y_region_1_spring, test_y_region_1_spring = before_modeling(x_region_1_spring, y_region_1_spring, 'minmax')
train_x_region_2_spring, test_x_region_2_spring, train_y_region_2_spring, test_y_region_2_spring = before_modeling(x_region_2_spring, y_region_2_spring, 'minmax')
train_x_region_3_spring, test_x_region_3_spring, train_y_region_3_spring, test_y_region_3_spring = before_modeling(x_region_3_spring, y_region_3_spring, 'minmax')
train_x_region_4_spring, test_x_region_4_spring, train_y_region_4_spring, test_y_region_4_spring = before_modeling(x_region_4_spring, y_region_4_spring, 'minmax')
train_x_region_5_spring, test_x_region_5_spring, train_y_region_5_spring, test_y_region_5_spring = before_modeling(x_region_5_spring, y_region_5_spring, 'minmax')
train_x_region_6_spring, test_x_region_6_spring, train_y_region_6_spring, test_y_region_6_spring = before_modeling(x_region_6_spring, y_region_6_spring, 'minmax')
train_x_region_7_spring, test_x_region_7_spring, train_y_region_7_spring, test_y_region_7_spring = before_modeling(x_region_1_spring, y_region_7_spring, 'minmax')
train_x_region_8_spring, test_x_region_8_spring, train_y_region_8_spring, test_y_region_8_spring = before_modeling(x_region_8_spring, y_region_8_spring, 'minmax')
train_x_region_9_spring, test_x_region_9_spring, train_y_region_9_spring, test_y_region_9_spring = before_modeling(x_region_9_spring, y_region_9_spring, 'minmax')
train_x_region_10_spring, test_x_region_10_spring, train_y_region_10_spring, test_y_region_10_spring = before_modeling(x_region_10_spring, y_region_10_spring, 'minmax')

Training shape :  (8563, 9) (8563, 1)
Test shape :  (2141, 9) (2141, 1)
Training shape :  (8563, 9) (8563, 1)
Test shape :  (2141, 9) (2141, 1)
Training shape :  (8563, 9) (8563, 1)
Test shape :  (2141, 9) (2141, 1)
Training shape :  (8563, 9) (8563, 1)
Test shape :  (2141, 9) (2141, 1)
Training shape :  (8563, 9) (8563, 1)
Test shape :  (2141, 9) (2141, 1)
Training shape :  (8563, 9) (8563, 1)
Test shape :  (2141, 9) (2141, 1)
Training shape :  (8563, 9) (8563, 1)
Test shape :  (2141, 9) (2141, 1)
Training shape :  (8563, 9) (8563, 1)
Test shape :  (2141, 9) (2141, 1)
Training shape :  (8563, 9) (8563, 1)
Test shape :  (2141, 9) (2141, 1)
Training shape :  (8563, 9) (8563, 1)
Test shape :  (2141, 9) (2141, 1)


In [19]:
train_x_name = [train_x_region_1_spring, train_x_region_2_spring, train_x_region_3_spring, train_x_region_4_spring, train_x_region_5_spring,
                train_x_region_6_spring, train_x_region_7_spring, train_x_region_8_spring, train_x_region_9_spring, train_x_region_10_spring]

test_x_name = [test_x_region_1_spring, test_x_region_2_spring, test_x_region_3_spring, test_x_region_4_spring, test_x_region_5_spring,
                test_x_region_6_spring, test_x_region_7_spring, test_x_region_8_spring, test_x_region_9_spring, test_x_region_10_spring]

train_y_name = [train_y_region_1_spring, train_y_region_2_spring, train_y_region_3_spring, train_y_region_4_spring, train_y_region_5_spring,
                train_y_region_6_spring, train_y_region_7_spring, train_y_region_8_spring, train_y_region_9_spring, train_y_region_10_spring]

test_y_name = [test_y_region_1_spring, test_y_region_2_spring, test_y_region_3_spring, test_y_region_4_spring, test_y_region_5_spring,
                test_y_region_6_spring, test_y_region_7_spring, test_y_region_8_spring, test_y_region_9_spring, test_y_region_10_spring]

num = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]

## torch에 x 담기
# 비어있는 torch 생성
def empty_torch(train_x):
  num_power = 10
  num_features = 7 ## surface_tp_train_stn, time_idx 제외하고 리스트로 들어갈 예정
  length = int(train_x.shape[0])
  size = (num_power, length, num_features)
  train_x_parallel = torch.empty(size)
  return train_x_parallel

# 비어있는 torch에 x 변수들 넣기
def dataload(x_name, num, x_parallel):
  ## train_x 변수 명에서 숫자가 몇인지를 보고, 이를 통해서 torch 위치를 지정하자
  region_index = num
  ## train_x 리스트로 제작:
  columns_feature = ["surface_tp_train_ta", "surface_tp_train_td", "surface_tp_train_hm", "surface_tp_train_ws", "surface_tp_train_rn", "surface_tp_train_re", "surface_tp_train_ww"]
  for i in range(len(x_name)):
    list_feature = x_name[columns_feature].iloc[i].to_list()
    ## 제작된 리스트를 순서대로 torch에 넣는다.
    for j in range(len(list_feature)):
      x_parallel[region_index, i, j] = list_feature[j]

## torch_y 담기...
# 비어있는 torch 생성
def empty_target(train_x):
  num_power = 10
  num_features = 1 ## surface_tp_train_stn, time_idx 제외하고 리스트로 들어갈 예정
  length = int(train_x.shape[0])
  size = (num_power, length, num_features)
  train_y_parallel = torch.empty(size)
  return train_y_parallel

# 비어있는 torch에 target 넣기
def target_torch(y_name, num, y_parallel):
  ## train_x 변수 명에서 숫자가 몇인지를 보고, 이를 통해서 torch 위치를 지정하자
  region_index = num
  ## train_x 리스트로 제작:
  for i in range(len(y_name)):
    tensor = y_name[i]
    ## 제작된 리스트를 순서대로 torch에 넣는다.
    for j in range(len(tensor)):
      y_parallel[region_index, i, j] = tensor[j]

In [20]:
## train_x 병렬화
train_x_parallel = empty_torch(train_x_region_1_spring)
for num in range(0,10,1):
  train_x = train_x_name[num]
  num = num
  dataload(train_x, num, train_x_parallel)

In [21]:
## test_x 병렬화
test_x_parallel = empty_torch(test_x_region_1_spring)
for num in range(0,10,1):
  test_x = test_x_name[num]
  num = num
  dataload(test_x, num, test_x_parallel)

In [22]:
## train_y 병렬화
train_y_parallel = empty_target(train_y_region_1_spring)
for num in range(0,10,1):
  train_y = train_y_name[num]
  num = num
  target_torch(train_y, num, train_y_parallel)

In [23]:
## test_y 병렬화
test_y_parallel = empty_target(test_y_region_1_spring)
for num in range(0,10,1):
  test_y = test_y_name[num]
  num = num
  target_torch(test_y, num, test_y_parallel)

In [25]:
train_x_parallel = Variable(train_x_parallel)
test_x_parallel = Variable(test_x_parallel)
train_y_parallel = Variable(train_y_parallel)
test_y_parallel = Variable(test_y_parallel)

In [38]:
train_x_parallel.shape

torch.Size([10, 8563, 1])

In [48]:
import torch
import torch.nn as nn
import torch.optim as optim

def mean_absolute_error(y_true, y_pred):
    return torch.mean(torch.abs(y_true - y_pred))

class LSTMModel(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size):
        super(LSTMModel, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.layer_1 = nn.Linear(hidden_size, 256)
        self.layer_2 = nn.Linear(256,256)
        self.layer_3 = nn.Linear(256,128)
        self.layer_out = nn.Linear(128, output_size)
        self.relu = nn.ReLU() #Activation Func



    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        out, (hn, cn) = self.lstm(x, (h0 , c0))
        out = self.layer_1(out) # first layer
        out = self.relu(out) # activation func relu
        out = self.layer_2(out)
        out = self.relu(out)
        out = self.layer_3(out)
        out = self.relu(out)
        out = self.layer_out(out) #Output layer
        return out

# 입력과 출력 크기 설정
input_size = 7
hidden_size = 128
num_layers = 2
output_size = 1

# 모델 인스턴스 생성
model = LSTMModel(input_size, hidden_size, num_layers, output_size)

# 손실 함수와 최적화 함수 설정
criterion = nn.L1Loss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# x 텐서와 y 텐서 생성 (임의의 값으로 초기화)
x_train = train_x_parallel  # torch.Size([10, 8563 , 7]) 10개의 지점에 대해서 7개의 변수가 존재하며 8563개의 시퀀의 데이터 존재
y_train = train_y_parallel  # torch.Size([10, 8563, 1])
x_valid = test_x_parallel  # torch.Size([10, 2141, 7])
y_valid = test_y_parallel  # torch.Size([10, 2141, 1])

# 학습
num_epochs = 100
for epoch in range(num_epochs):
    model.train()
    optimizer.zero_grad()
    outputs = model(x_train)
    loss = criterion(outputs, y_train)
    loss.backward()
    optimizer.step()

    # Validation
    model.eval()
    with torch.no_grad():
        valid_outputs = model(x_valid)
        valid_loss = criterion(valid_outputs, y_valid)
        valid_mae = mean_absolute_error(valid_outputs, y_valid)

    if (epoch + 1) % 10 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Train Loss: {loss.item():.4f}, Valid Loss: {valid_loss.item():.4f}, Valid MAE: {valid_mae.item():.4f}')

# 테스트
model.eval()
with torch.no_grad():
    predicted = model(x_valid)
    print("Predicted:", predicted)


Epoch [10/100], Train Loss: 9.0128, Valid Loss: 9.2925, Valid MAE: 9.2925
Epoch [20/100], Train Loss: 7.4694, Valid Loss: 7.4857, Valid MAE: 7.4857
Epoch [30/100], Train Loss: 7.4656, Valid Loss: 7.1123, Valid MAE: 7.1123
Epoch [40/100], Train Loss: 7.4775, Valid Loss: 7.3650, Valid MAE: 7.3650
Epoch [50/100], Train Loss: 7.4595, Valid Loss: 7.1473, Valid MAE: 7.1473
Epoch [60/100], Train Loss: 7.4455, Valid Loss: 7.2017, Valid MAE: 7.2017
Epoch [70/100], Train Loss: 7.4402, Valid Loss: 7.2188, Valid MAE: 7.2188
Epoch [80/100], Train Loss: 7.4301, Valid Loss: 7.1762, Valid MAE: 7.1762
Epoch [90/100], Train Loss: 7.4014, Valid Loss: 7.1866, Valid MAE: 7.1866
Epoch [100/100], Train Loss: 7.4193, Valid Loss: 7.3610, Valid MAE: 7.3610
Predicted: tensor([[[1.0568],
         [1.9099],
         [3.1008],
         ...,
         [7.9778],
         [7.4530],
         [7.9853]],

        [[1.0545],
         [1.9031],
         [3.0917],
         ...,
         [6.9597],
         [6.4921],
         